# Generieren von Daten
In diesem Notebook wird mithilfe des erstellten Modell's Musik generiert

In [1]:
# Needed imports
import numpy as np
import json
import os
from tensorflow import keras
from pathlib import Path

# Needed variabel
model_path = Path(os.path.abspath('')) / "model.h5"
datasets_folder = Path(os.path.abspath('')) / 'dataset'
mapping_path = datasets_folder / 'mappings.json'
melody_path = datasets_folder / 'melody.txt'
sequence_length = 64

2023-02-06 09:24:14.956463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 09:24:14.956489: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(mapping_path)
print(model_path)
print(datasets_folder)
print(melody_path)

/workspaces/Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze/demos/model.h5
/workspaces/Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze/demos/dataset
/workspaces/Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze/demos/dataset/mappings.json


## Lade das modell und die Mappings

In [3]:
model = keras.models.load_model(model_path)
model.summary()

2023-02-06 09:24:16.574025: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-06 09:24:16.574053: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-06 09:24:16.574076: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5193325dcd0b): /proc/driver/nvidia/version does not exist
2023-02-06 09:24:16.574344: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 18)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               281600    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 18)                4626      
Total params: 286,226
Trainable params: 286,226
Non-trainable params: 0
_________________________________________________________________


In [4]:
with open(mapping_path, 'r') as f:
    mappings = json.load(f)

print(mappings)

{'/': 0, '55': 1, '57': 2, '60': 3, '62': 4, '64': 5, '65': 6, '67': 7, '68': 8, '69': 9, '71': 10, '72': 11, '74': 12, '76': 13, '77': 14, '81': 15, '_': 16, 'r': 17}


In [5]:
start_symbols = ["/"] * sequence_length

## Erstellen des anfangsseeds
Jede Musikgenerierung benötigt eine Anfangsmellodie. Diese sollte am besten aus ganzen Takten bestehen

In [6]:
start_melodie = "72 _ _ _ _ _ 72 _ _ _ _ _ 72 _ 71 _"

## Generiere ein Lied

| Variable | Beschreibung |
| -------- | ------------ |
| `seed` | Start der Melodie, welche fortgeführt werden soll. Zusätzlich enthält dies die Noten, welches von der KI vorgeschlagen wird |
| `max_num_steps` | Die maximale anzahl an Schritten wo die KI neue Noten ausgeben soll |
| `max_squenece_length` | Wie viele Noten möchte man vom seed / start_melodie der KI vorgeben |
| `temperature` | Man kann hier defenieren, wie stark wir uns auf die Noten der "fixieren" möchten. Möchte man wirklich nur die Noten nehmen wo die KI am meisten denkt, dass diese verwendet werden soll oder ist man da mehr "flexibel"

Bei `temperature` ist die Abhänfigkeit so: </br>
 value => infinity --> Alle Wahrscheinlichkeiten, welche die KI ausspuckt, haben den gleichen Wert. Also wird die Note komplett zufällig ausgewählt</br>
 value => 0 --> Das was die höchste Wahrscheinlichkeit hat, wird definitiev verwendet</br>
 value => 1 --> Man verwendet genau die Wahrscheinlichkeiten, welche die KI ausgespuckt hat, um die Note / das Symbol festzulegen</br>

In [7]:
# Add start symbols to the start of the melody (this shows that the melody is starting, like the training data)
seed = start_symbols + start_melodie.split()
print(seed)

['/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '/', '72', '_', '_', '_', '_', '_', '72', '_', '_', '_', '_', '_', '72', '_', '71', '_']


In [8]:
# map the seed to integers
seed = [mappings[symbol] for symbol in seed]
print(seed)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 16, 16, 16, 16, 16, 11, 16, 16, 16, 16, 16, 11, 16, 10, 16]


In [9]:
# Start the melody generation
max_num_steps = 500     # max number of notes we want to generate
temperature = 1.0       # higher temperature, more random notes. lower temperature, more common notes
output_melody = start_melodie.split()

for _ in range(max_num_steps):
    # limit the seed to max_squenece_length
    # cut off the beginning of the seed if it is longer than max_sequence_length
    seed = seed[-sequence_length:]

    # One-Hot the seed
    onehot_seed = keras.utils.to_categorical(seed, num_classes=len(mappings))

    # Reshape the seed so that it can be fed to the model
    onehot_seed = onehot_seed[np.newaxis, ...]

    # Make a prediction
    probabilities = model.predict(onehot_seed)[0]

    # Apply the temperature to the probabilities
    predictions = np.log(probabilities) / temperature
    probabilities = np.exp(predictions) / np.sum(np.exp(predictions))   # normalize the probabilities. This is like applying softmax

    # Get the index of the predicted note
    index = np.random.choice(len(mappings), p=probabilities)

    # Update the seed
    seed.append(index)

    # Map the index to the note
    output_symbol = [k for k, v in mappings.items() if v == index][0]

    # Stop if we have reached the end of the melody
    if output_symbol == "/":
        break

    # Update the output melody
    output_melody.append(output_symbol)


2023-02-06 09:24:17.555555: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [10]:
print(output_melody)

['72', '_', '_', '_', '_', '_', '72', '_', '_', '_', '_', '_', '72', '_', '71', '_', '72', '_', '68', '_', '69', '_', '72', '_', '71', '_', '72', '_', '74', '_', '72', '_', '_', '_', '72', '_', '72', '_', '_', '_', '69', '_', '_', '_', '71', '_', '72', '_', '74', '_', '72', '_', '_', '_', '67', '_', '_', '_', '_', '_', '62', '_', '_', '_', '_', '_', '67', '_', '_', '_', '67', '_', '67', '_', '_', '_', '69', '_', '65', '_', '_', '_', '_', '_', '67', '_', '_', '_', '_', '_', '64', '_', '_', '_', '_', '_', '67', '_', '_', '_', '_', '_', '67', '_', '_', '_', '57', '_', '57', '_', '_', '_', '60', '_', '60', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']


In [11]:
# Save the melody
with open(melody_path, 'w') as f:
    json.dump(output_melody, f)